In [2]:
import matplotlib.pyplot as plt
plt.style.use('default')
import numpy as np
import re
import nltk

# Part-of-Speech Tagging (análise morfológica)

Part-of-speech tagging é a tarefa de identificar as classes morfológicas das palavras de uma frase. Nesta aula, usaremos o conceito de n-gramas para nos ajudar a identificar classes morfológicas automaticamente.

## Exercício 1
**Objetivo: lembrar-se do que é uma classe morfológica**

Nas frases abaixo, identifique os *substantivos*, os *verbos*, os *adjetivos* e os *advérbios*:

1. Hoje eu acordei serenamente e vi que era um dia lindo e calmo.
1. A mutação dos fungos é capaz de controlar a mente das pessoas!
1. Todo dia, o Sol da manhã vem e nos desafia!
1. Não adianta tentar fazer um sistema automático que faz alguma coisa que não entendemos o resultado!

## Exercício 2
**Objetivo: representar classes morfológicas no computador**

Quando falamos de *part-of-speech tagging*, queremos associar cada token de um texto a uma etiqueta que representa sua classe morfológica. A tarefa de rotular tokens pode ser realizada por pacotes prontos, por exemplo pelo `DefaultTagger` do `nltk`, que atribui o mesmo rótulo a qualquer token que seja recebido.

No exemplo abaixo, usamos o rótulo `N` (*noun*, ou "substantivo").

1. Como um par token+rótulo é representado?]
    - <font color=red> Uma tupla (token, rótulo) </font>


2. Na frase usada como exemplo, qual é a acurácia do rotulador?
    - <font color=red> Número de substantivos / quantidade de tokens = 2/9 </font>

In [3]:
from nltk.tag import DefaultTagger
default_tagger = DefaultTagger('N')
tokens = re.findall(r'\w+|[.,!?:]+', "uma frase qualquer: ótimo para dar aula!")
default_tagger.tag(tokens)

[('uma', 'N'),
 ('frase', 'N'),
 ('qualquer', 'N'),
 (':', 'N'),
 ('ótimo', 'N'),
 ('para', 'N'),
 ('dar', 'N'),
 ('aula', 'N'),
 ('!', 'N')]

## Exercício 3
**Objetivo: entender como funciona um banco de dados de tags**

Uma estratégia menos "inocente" que atribuir a mesma tag a todos os tokens é criar um grande banco de dados nos quais relacionamos palavras a tags. Por exemplo, a palavra "Brasil" é um substantivo, a palavra "andar" é um verbo, e assim por diante. Para construir esse dicionário, podemos usar um grande conjunto de frases rotuladas.

O corpus [macmorpho](http://www.nilc.icmc.usp.br/macmorpho/) tem uma série de anotações de *part-of-speech* na língua portuguesa.

1. Nos códigos abaixo, verifique como as tags são associadas a palavras no corpus macmorpho.
    - <font color=red> As tags são associadas as palavras por um "_" ao final da palavra seguido pelo símbolo do rótulo </font>


2. Como a instrução `re.findall` foi usada para extrair as tuplas de tokens/tags nesse corpus? A expressão regular usada poderia ser melhorada?

In [4]:
with open('./datasets/macmorpho-train.txt', 'r', encoding='utf-8') as f:
    print(f.readline())
    pass

Jersei_N atinge_V média_N de_PREP Cr$_CUR 1,4_NUM milhão_N na_PREP+ART venda_N da_PREP+ART Pinhal_NPROP em_PREP São_NPROP Paulo_NPROP ._PU



In [5]:
with open('./datasets/macmorpho-train.txt', 'r', encoding='utf-8') as f:
    tokens = [re.findall(r'([0-9,]+|\w+|[\W]+|\w+\$)\_([\w\+]+)', line) for line in f.readlines()]
print(tokens[0])

[('Jersei', 'N'), ('atinge', 'V'), ('média', 'N'), ('de', 'PREP'), ('Cr$', 'CUR'), ('1,4', 'NUM'), ('milhão', 'N'), ('na', 'PREP+ART'), ('venda', 'N'), ('da', 'PREP+ART'), ('Pinhal', 'NPROP'), ('em', 'PREP'), ('São', 'NPROP'), ('Paulo', 'NPROP'), (' .', 'PU')]


## Exercício 4
**Objetivo: avaliar um pos-tagger**

O método `accuracy` permite avaliar um tagger com base em um gabarito. Por exemplo, podemos usar a base de teste do *macmorpho* para fazer essa avaliação.

1. Com base no código abaixo, qual é a acurácia do tagger se simplesmente assumirmos que todas as palavras são substantivos?
    - <font color=red> 0.2048628435917992 </font>


2. E se a tag padrão for `V` (verbo), qual seria a acurácia?
    - <font color=red> 0.1105043924809248 </font>
    

In [6]:
default_tagger = DefaultTagger('N')

In [7]:
with open('./datasets/macmorpho-test.txt', 'r', encoding='utf-8') as f:
    tokens_test = [re.findall(r'([0-9,]+|\w+|[\W]+|\w+\$)\_([\w\+]+)', line) for line in f.readlines()]
default_tagger.accuracy(tokens_test)

0.2048628435917992

## Exercício 5
**Objetivo: treinar e avaliar um pos-tagger**

Uma maneira de encontrar quais palavras devem receber determinadas tags é treinar um dicionário em um grande banco de dados e simplesmente atribuir a cada palavra o rótulo que é mais comum a essa palavra, ou seja, usar a probabilidade da tag $t$ dado que sabemos a palavra $w$:

$$
P(t | w)
$$

Isso pode ser implementado no nltk usando:

In [8]:
from nltk.tag import UnigramTagger
unigram_tagger = UnigramTagger(tokens, backoff=default_tagger)
unigram_tagger.accuracy(tokens_test)

0.8922594787327679

1. Diferente do `sklearn`, fazemos o treinamento do `UnigramTagger` já quando ele é instanciado. Qual é o parâmetro que indica o banco de dados de treinamento?
    - <font color=red> primeiro parâmetro (train: tokens, test: tokens_test) </font>


2. O que significa `backoff`, e por que ele é importante?
    - <font color=red> para classificar uma palavra que não exista no vocabulário </font>
    
    
3. Qual é a acurácia do `UnigramTagger` com backoff no `macmorpho-test`?
    - <font color=red> 0.8922594787327679 </font>

## Exercício 6
**Objetivo: treinar e avaliar um pos-tagger baseado em bigramas**

Algumas palavras, como "Brasil", claramente pertencem a uma classe morfológica única. Outras, como "andar", podem assumir classes gramaticais diferentes dependendo do contexto ("eu moro no terceiro andar" / "vou andar até ali"). Para resolver essa questão, podemos usar *n-gramas* ao invés de palavras para encontrar a tag, isto é:

$$
P(t_n | w_n, w_{n-1}, ..., w_{n-N-1})
$$

Podemos implementar um tagger baseado em bi-gramas usando:

In [11]:
from nltk import NgramTagger
bigram_tagger = NgramTagger(n=2, train=tokens, backoff=unigram_tagger)
bigram_tagger.accuracy(tokens_test)

0.9103059319515846

Neste código:

1. Onde especificamos que o tagger deve conter bigramas?
    - <font color=red>No primeiro parâmetro da função NgramTagger (n=2)</font>


2. Qual é a importância do backoff nesse caso?
    - <font color=red>Caso apareça uma dupla de palavras inexistentes no vocabulário.</font>

## Exercício 7
**Objetivo: implementar e testar taggers com n-gramas**

Tomando por base os códigos anteriores, implemente e avalie um pos-tagger com trigramas e depois com tetragramas.

1. A acurácia aumenta significativamente quando aumentamos o contexto?
    - <font color=red>Não. A acurácia não aumenta significativamente.</font>


2. O que acontece com a acurácia se removermos o backoff?
    - <font color=red>O valor de acurácia cai drásticamente, ou seja, não há tantas repetições de 3 e 4 palavras.</font>

In [12]:
#Implementação de um pos-tagger com trigramas
trigram_tagger = NgramTagger(n=3, train=tokens, backoff=bigram_tagger)
trigram_tagger_ = NgramTagger(n=3, train=tokens)

In [13]:
#Implementação de um pos-tagger com tetragramas
tetragram_tagger = NgramTagger(n=4, train=tokens, backoff=trigram_tagger)
tetragram_tagger_ = NgramTagger(n=4, train=tokens)

In [14]:
print(unigram_tagger.accuracy(tokens_test))
print(bigram_tagger.accuracy(tokens_test))
print(trigram_tagger.accuracy(tokens_test))
print(tetragram_tagger.accuracy(tokens_test))

0.8922594787327679
0.9103059319515846
0.9106703368783392
0.909734096528062


In [15]:
print(trigram_tagger_.accuracy(tokens_test))
print(tetragram_tagger_.accuracy(tokens_test))

0.17991512168321439
0.13031120180744843


## Exercício 8
**Objetivo: testar o tagger em situações reais**

Verifique como seu tagger se comporta quando tenta rotular:

1. Uma frase que poderia ser usada normalmente na língua escrita

In [16]:
trigram_tagger.tag("ola mundo cruel!".split())

[('ola', 'N'), ('mundo', 'N'), ('cruel!', 'N')]

2. Uma frase com neologismos usados na Internet como "vc", "rsrsrs", etc.

In [17]:
trigram_tagger.tag("Ow, vc vai colar aqui em casa? Se tu for me avisa q horas vc chega pra eu abrir o portão pra vc.".split())

[('Ow,', 'N'),
 ('vc', 'N'),
 ('vai', 'V'),
 ('colar', 'V'),
 ('aqui', 'ADV'),
 ('em', 'PREP'),
 ('casa?', 'N'),
 ('Se', 'KS'),
 ('tu', 'PROPESS'),
 ('for', 'V'),
 ('me', 'PROPESS'),
 ('avisa', 'V'),
 ('q', 'N'),
 ('horas', 'N'),
 ('vc', 'N'),
 ('chega', 'V'),
 ('pra', 'PREP'),
 ('eu', 'PROPESS'),
 ('abrir', 'V'),
 ('o', 'ART'),
 ('portão', 'N'),
 ('pra', 'PREP'),
 ('vc.', 'N')]

3. Emoticons como ":)" ou ":-)".

In [18]:
trigram_tagger.tag("Parsa, acordei bem hoje e fui pra academia cedo. Mó feliz :)".split())

[('Parsa,', 'N'),
 ('acordei', 'N'),
 ('bem', 'ADV'),
 ('hoje', 'ADV'),
 ('e', 'KC'),
 ('fui', 'V'),
 ('pra', 'PREP'),
 ('academia', 'N'),
 ('cedo.', 'N'),
 ('Mó', 'N'),
 ('feliz', 'ADJ'),
 (':)', 'N')]

## Exercício 9
**Objetivo: salvar o tagger e carregar em outro contexto**

Usando `joblib`, podemos salvar nosso tagger para evitar ter que carregar toda a base de dados em outro contexto.



In [19]:
import joblib

joblib.dump(trigram_tagger, 'tagger.joblib')
tagger = joblib.load('tagger.joblib')
tagger.tag("esta alsdkfjasdljf uma frase, :-) cheia de coisas novas".split())

[('esta', 'PROADJ'),
 ('alsdkfjasdljf', 'N'),
 ('uma', 'ART'),
 ('frase,', 'N'),
 (':-)', 'N'),
 ('cheia', 'ADJ'),
 ('de', 'PREP'),
 ('coisas', 'N'),
 ('novas', 'ADJ')]

Usando o `joblib`, salve o melhor tagger que você encontrou nesta aula. Envie o arquivo `.joblib` para um colega e, ao receber um tagger de volta, teste-o.

# Código completo

In [20]:
import re
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk import NgramTagger
import joblib

with open('./datasets/macmorpho-train.txt', 'r', encoding='utf-8') as f:
    tokens = [re.findall(r'(\w+|[\W]+)\_([\w\+]+)', line) for line in f.readlines()]

default_tagger = DefaultTagger('N')
unigram_tagger = UnigramTagger(tokens, backoff=default_tagger)
bigram_tagger = NgramTagger(n=2, train=tokens, backoff=unigram_tagger)
trigram_tagger = NgramTagger(n=3, train=tokens, backoff=bigram_tagger)

with open('./datasets/macmorpho-test.txt', 'r', encoding='utf-8') as f:
    tokens_test = [re.findall(r'(\w+|[\W]+)\_([\w\+]+)', line) for line in f.readlines()]
    
print(trigram_tagger.accuracy(tokens_test))

joblib.dump(trigram_tagger, 'tagger.joblib')
tagger = joblib.load('tagger.joblib')

0.9111468663979414
